In [3]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3 import TD3

In [4]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (0, 0, 0)
car_name = "TD3"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [5]:
folder = '/mnt/d/donkeycar_model/TD3/'

In [6]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [7]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/gym/spaces/box.py:78: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


starting DonkeyGym env
Setting default: start_delay 5.0
Setting default: max_cte 8.0
Setting default: frame_skip 1
Setting default: cam_resolution (120, 160, 3)
Setting default: log_level 20
Setting default: host localhost
Setting default: steer_limit 1.0
Setting default: throttle_min 0.0
Setting default: throttle_max 1.0
loading scene mini_monaco


INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


In [8]:
for i in range(1000):
    if i == 0:
        model = TD3("CnnPolicy", env, verbose=1, learning_rate = 0.003, tau = 0.3, buffer_size=100000 )
        Log={"TestReward":[]}

    # training
    obs = env.reset()
    model.learn(total_timesteps=2500, progress_bar=0)
    model.save(f'{folder}{(i+1)*2500}.pth')

    # testing
    tem = 0
    for _ in range(20):
        obs = env.reset()
        while True:
            action, _states = model.predict(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            # print (reward, done)
            env.render()
            tem += reward
            if done: break
    Log["TestReward"].append(tem/20)
    print(f"testing {(i+1)*2500} : {tem/20}")
    np.save(f"{folder}/Log_TD3-{(i+1)*2500}.npy", Log)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 32.2     |
|    ep_rew_mean     | -97.2    |
| time/              |          |
|    episodes        | 4        |
|    fps             | 8        |
|    time_elapsed    | 14       |
|    total_timesteps | 129      |
| train/             |          |
|    actor_loss      | 6.54     |
|    critic_loss     | 61.3     |
|    learning_rate   | 0.003    |
|    n_updates       | 28       |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.4     |
|    ep_rew_mean     | -85.5    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 7        |
|    time_elapsed    | 23       |
|    total_timesteps | 179      |
| train/             |          |
|    actor_loss      | 32.9     |
|    critic_loss     | 24.9     |
|    learning_rate   | 0.003    |
|    n_updates       | 78       |
--------------

/home/user/miniconda3/envs/donkey/lib/python3.11/site-packages/stable_baselines3/common/utils.py:573: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  return th.as_tensor(obs, device=device)


testing 2500 : -89.91043786527402
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 31.8     |
|    ep_rew_mean     | -101     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 9        |
|    time_elapsed    | 13       |
|    total_timesteps | 127      |
| train/             |          |
|    actor_loss      | 25       |
|    critic_loss     | 19.3     |
|    learning_rate   | 0.003    |
|    n_updates       | 2426     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.4     |
|    ep_rew_mean     | -87.1    |
| time/              |          |
|    episodes        | 8        |
|    fps             | 8        |
|    time_elapsed    | 21       |
|    total_timesteps | 179      |
| train/             |          |
|    actor_loss      | 22.7     |
|    critic_loss     | 24.4     |
|    learning_rate   | 0.003    |
|    n_updates

KeyboardInterrupt: 

In [25]:
num = 2500
test_model = TD3.load(f"{folder}{num}.pth")
obs = env.reset()
tem = 0
while True:
    action, _states = test_model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    print (reward, done)
    env.render()
    tem += reward
    if done: break
print(tem)

-2.998000063609254 False
-2.996000146960592 False
-2.988252516481989 False
-2.976368277227353 False
-2.9605643978356135 False
-2.9553396394920233 False
-2.949712531415186 False
-2.935257243394032 False
-2.928813819390113 False
-2.9186634481591507 False
-2.9084236273894057 False
-2.8956246719029948 False
-2.885900864319368 False
-2.8796793156841205 False
-2.87381361877477 False
-2.8624104562289023 False
-2.852931294185593 False
-2.844178730780014 False
-2.8335825271924895 False
-2.8257316629326907 False
-2.8164834742643015 False
-2.807665913044905 False
-2.799064486499443 False
-2.790517916911467 False
-2.782119377639993 False
-2.7737750057532926 False
-2.7655738900422997 False
-2.7574309797007452 False
-2.749399250259917 False
-2.741521570595518 False
-2.733825109051007 False
-2.726236218715884 False
-2.718742302939417 False
-2.711417288335321 False
-2.7042039614721607 False
-2.697163426273773 False
-2.6902689575208285 False
-2.6834996392370662 False
-2.676838522305428 False
-2.6703522

In [30]:
env.close()